<a href="https://colab.research.google.com/github/Rachel-Eva/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from tqdm import tqdm

# 📁 Step 0: Upload PDF
uploaded = files.upload()
PDF_PATH = list(uploaded.keys())[0]  # Automatically get the uploaded file name

# 📄 Step 1: Load and split the PDF
loader = PyPDFLoader(PDF_PATH)
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(pages)

# 🧠 Step 2: Embed using lightweight HuggingFace model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embedding=embeddings)

# 🤖 Step 3: Use LLaMA 3.1 8B model via Ollama
llm = Ollama(model="llama3:8b")

# 🧾 Step 4: Prompt template
prompt_template = """
You are a helpful assistant. Answer based only on the provided context.

Context: {context}
Question: {question}

Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# 🔄 Step 5: QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


Saving CC_User_Agreement.pdf to CC_User_Agreement.pdf


In [ ]:
while True:
    question = input("Ask a question (or 'exit'): ")
    if question.lower() == "exit":
        break
    result = qa_chain({"query": question})
    print("\n🧠 Answer:", result["result"])

Ask a question (or 'exit'): what is the pdf about?

🧠 Answer: The PDF appears to be a set of guidelines or rules for an event, covering topics such as Safety and Security, and Photography and Recording. It provides instructions on what to do (e.g. follow safety instructions) and what not to do (e.g. refrain from bringing prohibited items).
Ask a question (or 'exit'): summarize the pdf

🧠 Answer: Here is a summary of the PDF:

**By attending the film festival, guests acknowledge that certain activities may involve risks. Coral Cloud Resort is not liable for injuries, losses, or damages incurred during participation.**

**Ticketing and Admissions:**

* Festival passes and event tickets are non-transferable and must be presented for entry.
* The resort encourages guests to use their app to view tickets and receive notifications of any changes to the event schedule.
* Lost or stolen paper tickets cannot be replaced.

**Event Modifications:**

* Coral Cloud Resort reserves the right to modi